# Logistic Regression using PySpark

Let's see an example of how to run a logistic regression with Python and Spark! This is documentation example, we will quickly run through this and then show a more realistic example, afterwards.

In [ ]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("Logistic Regression").getOrCreate()

In [ ]:
# check (try) if Spark session variable (spark) exists and print information about the Spark context
try:
    spark
except NameError:
    print("Spark session does not context exist. Please create Spark session first (run cell above).")
else:
    configurations = spark.sparkContext.getConf().getAll()
    for item in configurations: print(item)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
# Load training data
training = spark.read.format("libsvm").load("data/sample_libsvm_data.txt")

lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(training)

trainingSummary = lrModel.summary

In [ ]:
trainingSummary.predictions.show()

In [ ]:
# May change soon!
from pyspark.mllib.evaluation import MulticlassMetrics

In [ ]:
lrModel.evaluate(training)

In [ ]:
# Usually would do this on a separate test set!
predictionAndLabels = lrModel.evaluate(training)

In [ ]:
predictionAndLabels.predictions.show()

In [ ]:
predictionAndLabels = predictionAndLabels.predictions.select('label','prediction')

In [ ]:
predictionAndLabels.show()

## Evaluators

Evaluators will be a very important part of our pipline when working with Machine Learning, let's see some basics for Logistic Regression, useful links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [ ]:
# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

In [ ]:
acc = evaluator.evaluate(predictionAndLabels)

In [ ]:
acc

Okay let's move on see some more examples!

# Logistic Regression Code Along
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

In [ ]:
data = spark.read.csv('data/titanic.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

In [ ]:
data.columns

In [ ]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [ ]:
my_final_data = my_cols.na.drop()

### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [ ]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [ ]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [ ]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [ ]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [ ]:
from pyspark.ml.classification import LogisticRegression

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [ ]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [ ]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [ ]:
fit_model = pipeline.fit(train_titanic_data)

In [ ]:
results = fit_model.transform(test_titanic_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [ ]:
results.select('Survived','prediction').show()

In [ ]:
AUC = my_eval.evaluate(results)

In [ ]:
AUC

# Logistic Regression Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [ ]:
data = spark.read.csv('data/customer_churn.csv',inferSchema=True,
                     header=True)

In [ ]:
data.printSchema()

### Check out the data

In [ ]:
data.describe().show()

In [ ]:
data.columns

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
final_data = output.select('features','churn')

### Test Train Split

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol='churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()

### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

### Using AUC

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [ ]:
auc

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
new_customers = spark.read.csv('data/new_customers.csv',inferSchema=True,
                              header=True)

In [ ]:
new_customers.printSchema()

In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

In [ ]:
final_results = final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()

Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!

## Great Job!

## Stop The Spark Session

In [ ]:
# stop the underlying SparkContext.
try:
    spark
except NameError:
    print("Spark session does not context exist - nothing to stop.")
else:
    spark.stop()